# Numerical Data - Cleanup

<br>
**Author** Mary Letey <br>
**Date** 3/20/18 <br>
**File** numclean.ipynb <br>
**Purpose** Formating numerical (Bloomberg) data

<br><br>

In [17]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import datetime
%matplotlib inline 

In [64]:
def getDates(rowlist, dates):
    datestring = rowlist[1]
    datelist = datestring.split('/')
    year = int(datelist[2])
    if (int(year) > 20):
        year += 1900
    else:
        year += 2000
    datelist[2] = str(year)
    old_datelist = datelist
    datelist = ''
    for j in range(0,len(old_datelist)):
        datelist += old_datelist[j]
        if j < len(old_datelist)-1:
            datelist += '/'
    dateobject = datetime.datetime.strptime(datelist,'%m/%d/%Y')
    dates.append(dateobject)    
    return dates

In [66]:
# List of first names
file_path = "../datafiles/numerical-initial/hp-data.csv"
files = []
for file in open(file_path,"r"):
    files.append(file.split('\n')[0])

l = len(files)
print(l)
i = 6112
a = files[i]
b = a.split(',')
print(b[1])
print(datetime.datetime.strptime('3/19/2018','%m/%d/%Y'))

dates = []
spots = []
vols = []
avgs = []

for i in range(1,len(files)):
    row = files[i]
    rowlist = row.split(',')
    if rowlist != ['', '', '', '', '']:
        dates = getDates(rowlist, dates)
        spots.append(rowlist[2])
    
print(spots)
        

#         print(rowlist)

6113
2/6/98
2018-03-19 00:00:00
['23.2', '23.51', '23.48', '23.48', '23.62', '23.83', '24.65', '23.95', '24.04', '23.95', '23.69', '23.57', '23.35', '23.39', '23.52', '23.46', '22.13', '21.39', '21.17', '21.39', '21.4', '21.45', '21.29', '20.58', '20.41', '20.11', '19.92', '21.21', '21.43', '21.29', '22.48', '23.3', '23.32', '23.44', '23.81', '23.91', '23.32', '23.66', '23.81', '23.85', '23.7', '23.29', '23.46', '22.88', '22.92', '22.41', '22.05', '21.81', '21.84', '21.75', '21.53', '21.37', '21.32', '21.01', '21.15', '21.27', '21.23', '21.26', '21.25', '21.46', '21.42', '21.29', '20.92', '20.74', '20.85', '20.98', '21.3', '21.07', '20.97', '21.02', '21.07', '21.08', '21.41', '21.45', '21.43', '21.66', '21.25', '21.24', '21.34', '22.46', '22.12', '21.75', '21.57', '21.36', '21.36', '21.17', '21.23', '21.34', '21.42', '21.38', '21.44', '21.47', '21.44', '21.47', '21.55', '21.39', '21.5', '21.52', '21.67', '21.95', '22.12', '22.02', '21.96', '21.72', '21.55', '21.79', '21.71', '20.4', '2